In [45]:
# lstm for time series forecasting
from numpy import sqrt
from numpy import asarray
from pandas import read_csv
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import time
from datetime import datetime
from tensorflow import math
from tensorflow import reduce_mean
import tensorflow as tf
import sys

In [3]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    print('X: \n', X[0:5], '\n Y: \n', y[0:5])
    return asarray(X), asarray(y)

In [4]:
# load the dataset
df = read_csv("SolarTrainingData.csv", index_col=0)

In [71]:
# try to drop the day column for better results
df.head(10)

temp  feelslike   dew  humidity  precip  snow  snowdepth  windgust  \
197  25.0       14.1  11.4     55.86     0.0     0          0      18.9   
198  25.0       14.1  11.4     55.86     0.0     0          0      18.9   
199  25.0       14.1  11.4     55.86     0.0     0          0      18.9   
201  25.0       14.1  11.4     55.86     0.0     0          0      18.9   
202  25.0       14.1  11.4     55.86     0.0     0          0      18.9   
203  25.0       14.1  11.4     55.86     0.0     0          0      18.9   
204  25.0       14.1  11.4     55.86     0.0     0          0      18.9   
205  25.0       14.1  11.4     55.86     0.0     0          0      18.9   
206  25.0       14.1  11.4     55.86     0.0     0          0      18.9   
207  25.0       14.1  11.4     55.86     0.0     0          0      18.9   

     windspeed  winddir  ...  cloudcover  visibility  solarradiation  \
197       11.7     10.0  ...        97.5         9.9            18.0   
198       11.7     10.0  ...        97.5         9.9            18.0   
199       11.7     10.0  ...        97.5         9.9            18.0   
201       11.7     10.0  ...        97.5         9.9            18.0   
202       11.7     10.0  ...        97.5         9.9            18.0   
203       11.7     10.0  ...        97.5         9.9            18.0   
204       11.7     10.0  ...        97.5         9.9            18.0   
205       11.7     10.0  ...        97.5         9.9            18.0   
206       11.7     10.0  ...        97.5         9.9            18.0   
207       11.7     10.0  ...        97.5         9.9            18.0   

     solarenergy  uvindex  year  month  day  time  Percent Output  
197          0.1      0.0  2018      1    1   8.0        0.014921  
198          0.1      0.0  2018      1    1   8.0        0.015856  
199          0.1      0.0  2018      1    1   8.0        0.003378  
201          0.1      0.0  2018      1    1   8.0        0.017996  
202          0.1      0.0  2018      1    1   8.0        0.016960  
203          0.1      0.0  2018      1    1   8.0        0.010319  
204          0.1      0.0  2018      1    1   8.0        0.014286  
205          0.1      0.0  2018      1    1   8.0        0.016170  
206          0.1      0.0  2018      1    1   8.0        0.016490  
207          0.1      0.0  2018      1    1   8.0        0.016364  

[10 rows x 21 columns]

In [6]:
# retrieve the values
values = df.values.astype('float32')
# specify the window size
n_steps = 5
# split into samples
X, y = split_sequence(values, n_steps)
# reshape into [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 21))
# split into train/test
n_test = 100
X_train, X_test, y_train, y_test = X[:-n_test], X[-n_test:], y[:-n_test], y[-n_test:]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

X: 
 [array([[2.5000000e+01, 1.4100000e+01, 1.1400000e+01, 5.5860001e+01,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.8900000e+01,
        1.1700000e+01, 1.0000000e+01, 1.0416000e+03, 9.7500000e+01,
        9.8999996e+00, 1.8000000e+01, 1.0000000e-01, 0.0000000e+00,
        2.0180000e+03, 1.0000000e+00, 1.0000000e+00, 8.0000000e+00,
        1.4920635e-02],
       [2.5000000e+01, 1.4100000e+01, 1.1400000e+01, 5.5860001e+01,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.8900000e+01,
        1.1700000e+01, 1.0000000e+01, 1.0416000e+03, 9.7500000e+01,
        9.8999996e+00, 1.8000000e+01, 1.0000000e-01, 0.0000000e+00,
        2.0180000e+03, 1.0000000e+00, 1.0000000e+00, 8.0000000e+00,
        1.5855573e-02],
       [2.5000000e+01, 1.4100000e+01, 1.1400000e+01, 5.5860001e+01,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.8900000e+01,
        1.1700000e+01, 1.0000000e+01, 1.0416000e+03, 9.7500000e+01,
        9.8999996e+00, 1.8000000e+01, 1.0000000e-01, 0.0000000

In [77]:
# define model
# improvement area : try adding dropout
model = Sequential()
model.add(LSTM(100, activation='relu', kernel_initializer='he_normal', input_shape=(n_steps,21)))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1))
# compile the model
model.compile(optimizer='adam', loss=custom_eval, metrics=['mae'])

In [78]:
# fit the model
model.fit(X_train, y_train, epochs=350, batch_size=32, verbose=2, validation_data=(X_test, y_test))
# evaluate the model
mse, mae = model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f, MAE: %.3f' % (mse, sqrt(mse), mae))

Epoch 1/350
[[99.3 104.2 67 ... 16 15 0.55536]
 [71.7 71.7 60.7 ... 31 12 0.350204736]
 [68.2 68.2 35.8 ... 8 15 0.38728413]
 ...
 [69.8 69.8 28.8 ... 8 14 0.733061254]
 [76.9 76.9 58.8 ... 26 20 -0.00155490765]
 [93.4 96.8 66.5 ... 24 12 0.59]] Y actual: 
[[96 101.3 68.3 ... 16 14 0.662539661]
 [87.1 93.4 71.7 ... 16 11 0.40405798]
 [94 96.9 65.7 ... 14 19 0.0468253978]
 ...
 [103.2 103.2 57.8 ... 17 17 0.0354166664]
 [91.2 96 68.8 ... 8 17 0.173469394]
 [65 65 62.9 ... 21 8 0.00212244899]] Y actual: 
[[47.1 42.4 32.5 ... 12 9 0.135790333]
 [67.9 67.9 58.4 ... 11 10 0.118552878]
 [81.1 86.4 73.8 ... 15 9 0.192792788]
 ...
 [47.2 42.3 22.5 ... 11 20 -0.000317460304]
 [92.9 97.2 67.7 ... 13 14 0.540271521]
 [94.5 98.8 67.4 ... 4 14 0.0535626523]] Y actual: 
[[68.7 68.7 51.6 ... 15 12 0.436877817]
 [54.9 54.9 20.9 ... 3 15 0.531636357]
 [71.9 71.9 57.1 ... 9 16 0.0503472239]
 ...
 [86.3 86.2 60.4 ... 26 16 0.523303151]
 [65.1 65.1 33.9 ... 20 13 0.758558571]
 [84.3 84.6 61.2 ... 10 13 0.

[[66 66 45 ... 1 10 0.288115948]
 [75.1 75.1 31.1 ... 21 15 0.723013937]
 [96.4 97.3 61.7 ... 5 17 0.194976449]
 ...
 [100.6 101.9 61.1 ... 2 17 0.517536223]
 [63.2 63.2 30.3 ... 11 20 -0.000571428565]
 [102.2 104.6 62.6 ... 21 15 0.329513878]] Y actual: 
[[89.8 94.7 69.3 ... 17 12 0.603174627]
 [84.6 91.8 73.7 ... 13 10 0.188851357]
 [95.6 97.7 63.9 ... 23 12 0.572238803]
 ...
 [31.5 23 14.5 ... 1 17 0.00136054424]
 [70.2 70.2 44.2 ... 11 13 0.723650813]
 [68.7 68.7 43.8 ... 28 14 0.674582541]] Y actual: 
[[71 71 55.3 ... 9 13 0.136263743]
 [65.5 65.5 60.8 ... 23 10 0.0314479657]
 [80.3 78.3 17.3 ... 19 15 0.695373118]
 ...
 [82.9 89.2 73.7 ... 30 9 0.34930557]
 [63.8 63.8 56.5 ... 20 16 0.067428574]
 [86.9 92.1 70.4 ... 27 11 0.377826095]] Y actual: 
[[71.6 71.6 49.6 ... 22 20 -0.00208333344]
 [80.9 78.8 19.6 ... 19 14 0.709104478]
 [95.5 101.8 69.6 ... 15 16 0.548730135]
 ...
 [84.3 89.4 71.1 ... 27 10 0.174254313]
 [52.6 52.6 13.6 ... 12 13 0.625555575]
 [87.1 93.4 71.7 ... 16 11 0

[[89.3 95.1 70.5 ... 3 11 0.388761908]
 [76.9 76.9 62.1 ... 23 15 0.175121471]
 [98.5 100.9 63.5 ... 28 15 0.644717455]
 ...
 [99.6 102 63.4 ... 28 15 0.54389143]
 [88.1 93 69.7 ... 1 11 0.389819]
 [78.4 78.4 74.9 ... 4 15 -0.00226244354]] Y actual: 
[[78.7 78.7 64.9 ... 27 12 0.207673475]
 [54.2 54.2 35.9 ... 20 10 0.381418914]
 [73.6 73.6 71.6 ... 9 8 0.00627943501]
 ...
 [67.1 67.1 57.5 ... 11 9 0.0488226041]
 [73 73 17.8 ... 6 15 0.54592]
 [73.3 73.3 32 ... 21 14 0.776252329]] Y actual: 
[[26.7 18.1 5.5 ... 16 20 -0.00179591833]
 [71 71 55.3 ... 9 13 0.1375]
 [98.7 95.5 49.2 ... 1 17 0.387755096]
 ...
 [63.9 63.9 27.9 ... 25 14 0.5448]
 [92.1 98.8 70.9 ... 10 15 0.552769661]
 [69.8 69.8 28.8 ... 8 14 0.713134348]] Y actual: 
[[92.8 94 63 ... 5 19 0.197910443]
 [27.8 17.5 8.9 ... 16 14 0.119425677]
 [100.7 102.3 61.7 ... 28 18 0.41248]
 ...
 [74.2 74.2 40.2 ... 21 19 0.0913043469]
 [63.9 63.9 19.8 ... 22 13 0.716883123]
 [67.9 67.9 36.8 ... 11 14 0.160144925]] Y actual: 
[[49.2 45.8

[[78.4 78.4 61.8 ... 4 17 0.119365081]
 [90.4 93.1 66.1 ... 5 14 0.725231886]
 [96.4 99.6 65.4 ... 14 15 0.552989364]
 ...
 [85 89.6 70 ... 24 10 0.269230783]
 [69.8 69.8 28.8 ... 8 14 0.66176]
 [91.1 96.6 69.7 ... 28 11 0.64618057]] Y actual: 
[[90.2 95.3 69.4 ... 21 12 0.58293134]
 [104.6 104.1 56.3 ... 22 17 0.0614583343]
 [70.7 70.7 62.1 ... 23 12 0.1472653]
 ...
 [70.2 70.2 52.2 ... 11 12 0.27331081]
 [96.4 99.6 65.4 ... 14 15 0.504202902]
 [95.1 96.8 63.4 ... 24 13 0.55104]] Y actual: 
[[87.3 92.9 70.9 ... 8 10 0.186545461]
 [84.3 89.4 71.1 ... 27 10 0.08784]
 [89.7 97.2 72.2 ... 6 12 0.46976]
 ...
 [69.4 69.4 49.3 ... 15 15 0.282028973]
 [88.4 87.2 57.7 ... 10 15 0.515646279]
 [94.9 97.2 64.5 ... 5 18 0.338363647]] Y actual: 
[[102.1 102.5 58.9 ... 17 15 0.628888905]
 [80.7 78.6 18.4 ... 19 16 0.505882382]
 [96.2 101.4 68.2 ... 15 17 0.43378377]
 ...
 [90.7 94.3 67.4 ... 4 20 -0.00408163248]
 [64.2 64.2 61.5 ... 27 19 -0.00451388909]
 [87.1 86.9 60.1 ... 10 14 0.672979593]] Y ac

KeyboardInterrupt: 

In [76]:
# Need new function to evaluate RMSE based on just 'Percent Output'
# Current state: we have input tensors of shape (None [batch size], 21). Need to find a way to turn that into an "array of losses"
# of shape (None, 21). Then return that.
# Current step: figure out how to index through a tensor. Need to only focus on solar output
def custom_eval(y_actual, y_pred):
    #print(y_actual.get_shape())
    print_output = tf.print(y_actual, "Y actual: ")
    SE_Tensor = math.squared_difference(y_pred, y_actual)  #squared difference
    #print(y_actual)
    MSE = reduce_mean(SE_Tensor)
    RMSE = tf.math.sqrt(MSE)
    return MSE

In [ ]:
# make a prediction
#row = asarray([18024.0, 16722.0, 14385.0, 21342.0, 17180.0]).reshape((1, n_steps, 1))
#yhat = model.predict(row)
#print('Predicted: %.3f' % (yhat))